<a href="https://colab.research.google.com/github/OdeliaAhdout/Drug-dataset/blob/main/Drugs25_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import sklearn
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.optim import AdamW

In [14]:
X_train = pd.read_csv('drive/My Drive/Colab Notebooks/drugs25/data25/X_train.csv',index_col = [0])
X_test = pd.read_csv('drive/My Drive/Colab Notebooks/drugs25/data25/X_test.csv',index_col = [0])
#y_train = pd.read_csv('drive/My Drive/Colab Notebooks/drugs25/data25/y_train.csv',index_col = [0])
#y_test = pd.read_csv('drive/My Drive/Colab Notebooks/drugs25/data25/y_test.csv',index_col = [0])

In [17]:
X_test = X_test['review'].astype(str).tolist()
X_train = X_train['review'].astype(str).tolist()
#y_train = y_train['rating'].tolist()
#y_test = y_test['rating'].tolist()

In [7]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

# Load the pre-trained DistilBERT tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [1]:
# Tokenize the training and test data
train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(X_test, padding=True, truncation=True, max_length=128, return_tensors='pt')

NameError: name 'tokenizer' is not defined

In [20]:
y_train_tensor = torch.tensor(y_train)
y_test_tensor = torch.tensor(y_test)

In [21]:
from torch.utils.data import DataLoader, RandomSampler

# Create datasets
train_dataset = TensorDataset(train_encodings["input_ids"], train_encodings["attention_mask"], y_train_tensor)
test_dataset = TensorDataset(test_encodings["input_ids"], test_encodings["attention_mask"], y_test_tensor)

# Create data loaders
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)


In [22]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(set(y_train)))
optimizer = AdamW(model.parameters(), lr=5e-5)

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# Training setup
epochs = 3
accumulation_steps = 4  # Number of steps to accumulate gradients

model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    epoch_loss = 0

    for step, batch in enumerate(train_loader):
        # Move batch to the device
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / accumulation_steps  # Scale loss for accumulation
        epoch_loss += loss.item()

        # Backward pass
        loss.backward()

        # Perform optimizer step after `accumulation_steps` steps
        if (step + 1) % accumulation_steps == 0 or (step + 1) == len(train_loader):
            optimizer.step()
            optimizer.zero_grad()  # Reset gradients

        if (step + 1) % 10 == 0:  # Log progress every 10 steps
            print(f"Step {step + 1}, Loss: {loss.item():.4f}")

    print(f"Epoch Loss: {epoch_loss / len(train_loader):.4f}")

Epoch 1/3
